In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import cv2 as cv
import os
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/ForAllOneK/Original'

In [ ]:
if not os.path.isdir(os.path.join(path, 'kmeans')):
  os.mkdir(os.path.join(path, 'kmeans'))
kmeans_path = os.path.join(path, 'kmeans') 

In [ ]:
def KMEANS(image, n_clusters):
  X = image.reshape(-1, 3)
  X = X/255.0
  km = KMeans(n_clusters=n_clusters)
  km.fit(X)
  clusters = km.cluster_centers_
  clusters = clusters[km.labels_]
  image_seg = clusters.reshape(image.shape)
  return image_seg

In [ ]:
data = pd.read_csv(os.path.join(path,'FinalKThresh2.csv'))
data.count()

file_name    1000
k_val        1000
cost_val     1000
dtype: int64

In [ ]:
data.iloc

In [ ]:
for file_name, k_val, cost_val in data.iloc:
  if os.path.isdir(os.path.join(kmeans_path, file_name)):
    continue
  if os.path.isfile(os.path.join(kmeans_path, file_name)):
    continue
  if not os.path.isdir(os.path.join(kmeans_path, file_name)):
    image = cv.imread(os.path.join(path,file_name))
    image_g = cv.GaussianBlur(image,(5,5),0)
    image_g = cv.GaussianBlur(image_g, (5,5), 0)
    image_g = cv.GaussianBlur(image_g, (5,5), 0)
    km_img = KMEANS(image_g, k_val)*255.0
    cv.imwrite(os.path.join(kmeans_path, file_name), km_img)
    print(file_name,"with K_val",k_val, "saved in kmeans_path")